In [2]:
import pandas as pd

Note: Stack Exchange datasets can be queried online: https://data.stackexchange.com/stackoverflow/query/new

# Posts

Stack Overflow has different types of posts, currently up to 8 types. Our work focuses on `Questions (Id=1)` and `Answers (Id=2)`. `Posts.csv` contains only answer and questions posts from the SOTorrent22 dataset.

In [3]:
df_posts = pd.read_csv('data/database_export/Posts.tsv', sep='\t')

In [4]:
display(df_posts.columns)

Index(['Id', 'PostTypeId', 'ParentId', 'CreationDate', 'DeletionDate',
       'LastEditDate', 'Tags', 'AnswerCount', 'CommentCount'],
      dtype='object')

In [5]:
display(df_posts.head())

,Id,PostTypeId,ParentId,CreationDate,DeletionDate,LastEditDate,Tags,AnswerCount,CommentCount
0,4,1,NaN,2008-07-31 21:42:52,NaN,2021-02-26 03:31:15,<c#><floating-point><type-conversion><double><...,13.0,4
1,6,1,NaN,2008-07-31 22:08:08,NaN,2021-01-29 18:46:45,<html><css><internet-explorer-7>,7.0,0
2,7,2,4.0,2008-07-31 22:17:57,NaN,2019-10-21 14:03:54,NaN,NaN,0
3,9,1,NaN,2008-07-31 23:40:59,NaN,2022-03-25 01:14:33,<c#><.net><datetime>,70.0,10
4,11,1,NaN,2008-07-31 23:55:37,NaN,2017-06-04 15:51:19,<c#><datetime><time><datediff><relative-time-s...,40.0,3


In [6]:
# Convert CreationDate to datetime
df_posts.CreationDate = pd.to_datetime(df_posts.CreationDate)
df_posts.dtypes

Id                       int64
PostTypeId               int64
ParentId               float64
CreationDate    datetime64[ns]
DeletionDate           float64
LastEditDate            object
Tags                    object
AnswerCount            float64
CommentCount             int64
dtype: object

In [7]:
nu_posttypes = df_posts.PostTypeId.value_counts()
display(nu_posttypes)

PostTypeId
2    33520483
1    22634239
Name: count, dtype: int64

In [8]:
print(f"Our SOTorrent data set has a total number of {df_posts.shape[0]:,} of posts.")

Our SOTorrent data set has a total number of 56,154,722 of posts.


In [9]:
print(f"This comprises {nu_posttypes[1]:,} questions and {nu_posttypes[2]:,} answers.")

This comprises 22,634,239 questions and 33,520,483 answers.


In [10]:
# Save df_posts to feather file
df_posts.to_feather('data/feather_files/Posts.feather')

# PostHistory

The `PostHistory` table contains the history of all posts on Stack Overflow. It includes information about the post's creation, edits, and deletions. The table on Stack Exchange has 22 columns, including the `PostHistoryTypeId` column, which indicates the type of history event. The `PostHistoryTypeId` column on Stack Exchange has 14 unique values, but we are only interested in the following types: `2`=`Initial Body` and `5`=`Edit Body`.
Every entry in the PosHistory table can have a commit message that describes the change made to the post, which we omitted here but which can be retrieved from the original dataset on Stack Exchange (see link at the top).

However, we will not use this table of Stack Exchange, since the SOTorrent data contains a more concise version of the `PostHistory` table, which is the `PostVersion` table. This table links together the various other tables in the SOTorrent dataset.

# PostVersion

The `SOTorrent` dataset added a dedicated `PostVersion` table for storing only edits made to the content of posts or when a post is rollbacked. For instance, if a question post is edited and new tags are added (i.e., `PostHistoryTypeId=6`) to the question, the edit is captured in the PostHistory table but not in the PostVersion table.

This means that the only PostHistory types in the PostVersion table are (2, 5, 8) which corresponds to `Initial Body`, `Edit Body` and `Rollback Body` respectively.

The `PostHistory` and `PostVersion` table are linked via the `Id` field of the `PostHistory` table. The `PostHistoryId` field of the PostVersion tables links the table to the `PostHistory` table. For instance, if you want to know the `User` that created a version of a post, you can get that information from the `PostHistory`table since the two tables are linked.

In [11]:
df_postversion = pd.read_csv("data/database_export/PostVersion.tsv", sep='\t')

In [12]:
df_postversion.CreationDate = pd.to_datetime(df_postversion.CreationDate)
df_postversion.PredPostHistoryId = df_postversion.PredPostHistoryId.fillna(0).astype(int)
df_postversion.SuccPostHistoryId = df_postversion.SuccPostHistoryId.fillna(0).astype(int)

In [13]:
df_postversion.dtypes

Id                            int64
PostId                        int64
PostTypeId                    int64
PostHistoryId                 int64
PostHistoryTypeId             int64
CreationDate         datetime64[ns]
PredPostHistoryId             int64
SuccPostHistoryId             int64
MostRecentVersion             int64
dtype: object

In [14]:
display(df_postversion)

,Id,PostId,PostTypeId,PostHistoryId,PostHistoryTypeId,CreationDate,PredPostHistoryId,SuccPostHistoryId,MostRecentVersion
0,1,13646426,1,32556579,2,2012-11-30 13:41:44,0,0,1
1,2,25989369,1,74075424,2,2014-09-23 07:29:34,0,0,1
2,3,37759745,1,120182375,2,2016-06-11 03:54:15,0,120194969,0
3,4,61414546,1,220036678,2,2020-04-24 17:56:14,0,220073195,0
4,5,37759745,1,120194969,5,2016-06-11 11:08:04,120182375,0,1
...,...,...,...,...,...,...,...,...,...
89524315,89524316,45760362,2,154260519,2,2017-08-18 15:39:00,0,0,1
89524316,89524317,45760364,2,154260527,2,2017-08-18 15:39:04,0,0,1
89524317,89524318,45760365,2,154260528,2,2017-08-18 15:39:04,0,154260951,0
89524318,89524319,45760365,2,154260951,5,2017-08-18 15:44:40,154260528,154261261,0


In [15]:
df_postversion.to_feather("data/feather_files/PostVersion.feather")

# PostVersionData

In [16]:
df_postversiondata = pd.read_csv('data/database_export/PostVersionData.tsv', sep='\t')

In [17]:
display(df_postversiondata.columns)

Index(['Id', 'PostId', 'PostVersionId', 'PostHistoryId',
       'RootPostBlockVersionId', 'PostBlockVersionId', 'Label'],
      dtype='object')

In [18]:
df_postversiondata.dtypes

Id                        int64
PostId                    int64
PostVersionId             int64
PostHistoryId             int64
RootPostBlockVersionId    int64
PostBlockVersionId        int64
Label                     int64
dtype: object

In [19]:
df_postversiondata.to_feather('data/feather_files/PostVersionData.feather')

# PostBlockVersion

Every `Post` consists of text blocks (`PostBlockTypeId=1`) and code blocks (`PostBlockTypeId=2`). The `PostBlockVersion` table contains the history of all text and code blocks of posts on Stack Overflow. It includes information about the block's creation, edits, and deletions.

In [20]:
df_postblockversion = pd.read_csv('data/database_export/PostBlockVersion.tsv', sep='\t')

In [21]:
display(df_postblockversion.columns)

Index(['Id', 'PostBlockTypeId', 'PostId', 'PostHistoryId',
       'PredPostBlockVersionId', 'PredPostHistoryId', 'RootPostBlockVersionId',
       'RootPostHistoryId', 'LineCount', 'MostRecentVersion'],
      dtype='object')

In [22]:
nu_postblocktypes = df_postblockversion.PostBlockTypeId.value_counts()
display(nu_postblocktypes)

PostBlockTypeId
1    167043851
2    109547454
Name: count, dtype: int64

In [23]:
print(f"Our SOTorrent22 data set has a total number of {df_postblockversion.shape[0]:,} of post block versions.")

Our SOTorrent22 data set has a total number of 276,591,305 of post block versions.


In [24]:
print(f"This comprises {nu_postblocktypes[1]:,} text block versions and {nu_postblocktypes[2]:,} code blocks versions.")

This comprises 167,043,851 text block versions and 109,547,454 code blocks versions.


In [25]:
# Save df_postblockversion to feather file
df_postblockversion.to_feather('data/feather_files/PostBlockVersion.feather')

# CodeBlockVersion

The `CodeBlockVersion` table contains the language of each code block, detect with Guesslang.

In [26]:
df_codeblockversion = pd.read_csv('data/database_export/CodeBlockVersion.tsv', sep='\t')

In [27]:
display(df_codeblockversion.columns)

Index(['Id', 'PostBlockVersionId', 'RootPostBlockVersionId', 'Language'], dtype='object')

In [28]:
# Save df_codeblockversion to feather file
df_codeblockversion.to_feather('data/feather_files/CodeBlockVersion.feather')

# SecurityRelevantCommits

The `SR_Commits` table contains the security-relevance classification of each commit based on the commit message of the `PostVersion`.

In [29]:
df_sr_commits = pd.read_csv('data/database_export/SecurityRelevantCommits.tsv', sep='\t', dtype={'Id':int, 'PostVersionId':int, 'IsRelevant':int, 'PostId':int})

In [30]:
display(df_sr_commits.columns)

Index(['PostId', 'PostVersionId', 'IsRelevant'], dtype='object')

In [31]:
nu_sr_commits = df_sr_commits.IsRelevant.value_counts()
display(nu_sr_commits)

IsRelevant
-1    64036788
 0     5848545
 1      777264
Name: count, dtype: int64

In [32]:
# Save df_sr_commits to feather file
df_sr_commits.to_feather('data/feather_files/SecurityRelevantCommits.feather')

# SecurityRelevantComments

In [33]:
df_sr_comments = pd.read_csv('data/database_export/SecurityRelevantComments.tsv', sep='\t')

In [34]:
display(df_sr_comments.columns)

Index(['Id', 'CommentId', 'PostId', 'CommentDate', 'IsRelevant'], dtype='object')

In [35]:
nu_sr_comments = df_sr_comments.IsRelevant.value_counts()
display(nu_sr_comments)

IsRelevant
0    45095570
1     3991533
Name: count, dtype: int64

In [36]:
# Save df_sr_comments to feather file
df_sr_comments.to_feather('data/feather_files/SecurityRelevantComments.feather')

# CommonMark Edits

The `CommonMarkEdits` table contains the `PostHistoryId` of each edit due to the CommonMark transformation of StackOverflow posts in June 2020.

In [37]:
df_commonmarkedits = pd.read_csv('data/database_export/PostHistoryId_CommonMark.tsv', sep='\t')
df_commonmarkedits.CreationDate = pd.to_datetime(df_commonmarkedits.CreationDate)
display(df_commonmarkedits)

,Id,CreationDate
0,224508120,2020-06-20 09:12:55
1,224508121,2020-06-20 09:12:55
2,224508122,2020-06-20 09:12:55
3,224508123,2020-06-20 09:12:55
4,224508124,2020-06-20 09:12:55
...,...,...
338617,224892323,2020-06-20 09:12:55
338618,224892324,2020-06-20 09:12:55
338619,224892325,2020-06-20 09:12:55
338620,224892326,2020-06-20 09:12:55


In [38]:
# Save df_commonmarkedits to feather file
df_commonmarkedits.to_feather('data/feather_files/PostHistoryId_CommonMark.feather')

# Tags September 2023

In [39]:
df_tags = pd.read_csv('data/database_export/Tags_September_2023.csv', parse_dates=True)

In [40]:
display(df_tags.columns)

Index(['QuestionId', 'Tags', 'CreationDate'], dtype='object')

In [41]:
display(df_tags.head())

,QuestionId,Tags,CreationDate
0,4,<c#><floating-point><type-conversion><double><...,2008-07-31 21:42:52
1,6,<html><css><internet-explorer-7>,2008-07-31 22:08:08
2,9,<c#><.net><datetime>,2008-07-31 23:40:59
3,11,<c#><datetime><time><datediff><relative-time-s...,2008-07-31 23:55:37
4,13,<html><browser><timezone><user-agent><timezone...,2008-08-01 00:42:38


In [42]:
# Save df_tags to feather file
df_tags.to_feather('data/feather_files/Tags_September_2023.feather')